# This code cleans the Taxi Trip data 

It removes duplicates, trips under 2 miles and trips over 1 day
It then exports the remaining data to a csv. Appropriate for 1 quarter of data at a time. 

In [1]:
import pandas as pd
from sodapy import Socrata
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.dates as mdates
import re

In [2]:
taxis = pd.read_csv('data/Chicago/Taxi_Trips_Q1.csv') #reads taxi trip information as taxis

In [3]:
taxis

,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,...,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location,Community Areas
0,f9d6229f00e2cd13206b24367f6a0344aad3d189,e1d51d08cb24e771a915f9058577f158307344815ca309...,03/31/2018 11:45:00 PM,03/31/2018 11:45:00 PM,252.0,0.60,1.703108e+10,1.703108e+10,8.0,8.0,...,8.75,Credit Card,Flash Cab,41.892073,-87.628874,POINT (-87.6288741572 41.8920726347),41.899156,-87.626211,POINT (-87.6262105324 41.8991556134),37.0
1,169553ad85411a2db0f488a523a1325a4140a847,30299139378a27a2bab44e84896e4b62d307097fa327ea...,03/31/2018 11:45:00 PM,03/31/2018 11:45:00 PM,117.0,0.20,1.703184e+10,1.703184e+10,24.0,24.0,...,4.75,Cash,Flash Cab,41.898306,-87.653614,POINT (-87.6536139825 41.8983058696),41.898306,-87.653614,POINT (-87.6536139825 41.8983058696),25.0
2,4be267085d9a40cab0fb71ef7a8bb557c6b4cbc7,ab31561a5548b1d49b0352bb8d00d18c0e136d5f674c1c...,03/31/2018 11:45:00 PM,04/01/2018 12:00:00 AM,522.0,1.70,1.703183e+10,1.703108e+10,28.0,8.0,...,9.25,Cash,Flash Cab,41.885281,-87.657233,POINT (-87.6572331997 41.8852813201),41.890922,-87.618868,POINT (-87.6188683546 41.8909220259),29.0
3,c1d99e1492049dd59b8dea876e2aeb904c699da4,7abac4926ff296e6403f9929f4527d9dd181e54e72f720...,03/31/2018 11:45:00 PM,03/31/2018 11:45:00 PM,13.0,0.00,NaN,NaN,7.0,7.0,...,4.25,Cash,Flash Cab,41.922686,-87.649489,POINT (-87.6494887289 41.9226862843),41.922686,-87.649489,POINT (-87.6494887289 41.9226862843),68.0
4,c15aa95b80c88c5f37d8134c70471be379bea040,020ac8feafd80a884d635ff8ffced25759473215d80a58...,03/31/2018 11:45:00 PM,03/31/2018 11:45:00 PM,427.0,2.50,1.703128e+10,1.703108e+10,28.0,8.0,...,10.25,Credit Card,Flash Cab,41.885300,-87.642808,POINT (-87.6428084655 41.8853000224),41.890922,-87.618868,POINT (-87.6188683546 41.8909220259),29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205246,7954dccd98d825b08664b6c590ecacac5cc14c29,0fc133469cf36857461460c9d18080fd979092f39f8d50...,01/01/2018 12:00:00 AM,01/01/2018 04:30:00 PM,58940.0,0.00,1.703108e+10,NaN,8.0,NaN,...,3.25,Cash,City Service,41.895033,-87.619711,POINT (-87.6197106717 41.8950334495),NaN,NaN,NaN,37.0
5205247,18b5ad526d36ee4c527d0ab849a3b5da230bd748,5e8fa091162e91db78f1256e485b80056e45352875fb82...,01/01/2018 12:00:00 AM,01/01/2018 12:15:00 AM,1073.0,4.95,NaN,NaN,NaN,NaN,...,21.00,Credit Card,Chicago Carriage Cab Corp,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5205248,222a5f4425c4077742e96e50529612c732dfb720,90bce85c296fc2baf956779a18b99da6d0227abacd2e8d...,01/01/2018 12:00:00 AM,01/01/2018 12:00:00 AM,360.0,0.40,NaN,NaN,NaN,NaN,...,6.25,Cash,Sun Taxi,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5205249,495523234a6b663b3e6cd445b912f3366c01e535,42560393a9c9b9ae28339f4b5aec77fd89bd49916ad541...,01/01/2018 12:00:00 AM,01/01/2018 12:00:00 AM,60.0,0.15,NaN,NaN,NaN,NaN,...,3.75,Cash,Patriot Taxi Dba Peace Taxi Associat,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
taxis.shape

(5205251, 24)

In [5]:
taxis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5205251 entries, 0 to 5205250
Data columns (total 24 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   Trip ID                     object 
 1   Taxi ID                     object 
 2   Trip Start Timestamp        object 
 3   Trip End Timestamp          object 
 4   Trip Seconds                float64
 5   Trip Miles                  float64
 6   Pickup Census Tract         float64
 7   Dropoff Census Tract        float64
 8   Pickup Community Area       float64
 9   Dropoff Community Area      float64
 10  Fare                        float64
 11  Tips                        float64
 12  Tolls                       float64
 13  Extras                      float64
 14  Trip Total                  float64
 15  Payment Type                object 
 16  Company                     object 
 17  Pickup Centroid Latitude    float64
 18  Pickup Centroid Longitude   float64
 19  Pickup Centroid Locat

In [6]:
taxis.describe()

,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,Fare,Tips,Tolls,Extras,Trip Total,Pickup Centroid Latitude,Pickup Centroid Longitude,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Community Areas
count,5.204841e+06,5.205085e+06,3.497162e+06,3.478743e+06,4.759280e+06,4.660420e+06,5.204999e+06,5.204999e+06,3.842069e+06,5.204999e+06,5.204999e+06,4.760886e+06,4.760886e+06,4.674605e+06,4.674605e+06,4.760877e+06
mean,7.754611e+02,3.508914e+00,1.703141e+10,1.703138e+10,2.553758e+01,2.236421e+01,1.278917e+01,1.605342e+00,2.628284e-03,9.739118e-01,1.548806e+01,4.189847e+01,-8.766058e+01,4.189884e+01,-8.765303e+01,4.035277e+01
std,1.368468e+03,5.596071e+00,3.450079e+05,3.336911e+05,2.004915e+01,1.758785e+01,2.465707e+01,2.850872e+00,4.541559e-01,1.122799e+01,2.873508e+01,3.915292e-02,7.703650e-02,3.826702e-02,5.924590e-02,1.408310e+01
min,0.000000e+00,0.000000e+00,1.703101e+10,1.703101e+10,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.166014e+01,-8.791362e+01,4.166014e+01,-8.791362e+01,1.000000e+00
25%,3.120000e+02,7.000000e-01,1.703108e+10,1.703108e+10,8.000000e+00,8.000000e+00,5.750000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.000000e+00,4.188099e+01,-8.764949e+01,4.188099e+01,-8.765600e+01,3.700000e+01
50%,5.400000e+02,1.300000e+00,1.703132e+10,1.703128e+10,2.800000e+01,2.400000e+01,7.750000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.500000e+00,4.189197e+01,-8.763275e+01,4.189204e+01,-8.763275e+01,3.700000e+01
75%,9.000000e+02,3.290000e+00,1.703184e+10,1.703184e+10,3.200000e+01,3.200000e+01,1.275000e+01,2.000000e+00,0.000000e+00,1.000000e+00,1.500000e+01,4.190022e+01,-8.762519e+01,4.190586e+01,-8.762621e+01,3.800000e+01
max,8.634000e+04,1.531600e+03,1.703198e+10,1.703198e+10,7.700000e+01,7.700000e+01,9.178000e+03,8.000000e+02,5.928000e+02,6.670660e+03,9.178000e+03,4.202122e+01,-8.753490e+01,4.202122e+01,-8.753490e+01,7.700000e+01


In [30]:
taxis.loc[(taxis.duplicated(subset=['Trip ID'], keep='first') == True)] #checking for duplicate ids

,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,...,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location,Community Areas


In [12]:
#taxis_clean = taxis_clean.loc[(taxis_clean.duplicated(subset=['Trip ID'], keep='first') == False)] 
#removes duplicate values of trip id

In [32]:
#filters out trips shorter than 2 miles and longer than 1 day
taxis_clean = taxis.loc[(taxis['Trip Miles'] >= 2) & (taxis['Trip Seconds'] <= 86400)] 
taxis_clean.shape

(1882150, 24)

In [36]:
taxis_clean.to_csv('data/Chicago/cleaned/taxis_clean_Q1.csv', index=False)
#exports clean dataset to csv

In [34]:
taxis_clean['Trip Start Timestamp'].min()

'01/01/2018 01:00:00 AM'

In [35]:
taxis_clean['Trip End Timestamp'].max()

'04/01/2018 12:30:00 AM'

In [37]:
#exports sample csv
taxi10000 = taxis_clean.head(10000)
taxi10000.to_csv('data/Chicago/cleaned/taxi10000.csv', index=False)